<a href="https://colab.research.google.com/github/shruthireddyrekula/Image_classification_cifar10_tf/blob/main/CIFAR10_RESNET50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()


In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
x_train=x_train/255
x_test=x_test/255

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train,num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test,num_classes=10)

In [ ]:
base_model = ResNet50(include_top = False, weights='imagenet',input_shape=(32,32,3))
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)
x= base_model.output
x = Flatten()(x)
x= Dense(256,activation='relu')(x)
predictions = Dense(10,activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input,outputs = predictions)

In [ ]:
model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Learning rate schedule
def lr_schedule(epoch):
    if epoch < 150:
        return 0.001
    elif epoch < 225:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
model.fit(datagen.flow(x_train, y_train, batch_size=64),
          epochs=100,
          validation_data=(x_test, y_test),
          callbacks=[lr_scheduler])

In [ ]:
test_loss, test_acc = model.evaluate(x_test,y_test)
print('Test Loss:', test_loss )
print('Test Accuracy:', test_acc)